In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 10.1MB/s 


In [0]:
from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler(feature_range=(200,500))

In [0]:
import pandas as pd
import numpy as np


In [0]:
link = 'https://drive.google.com/open?id=1EUwJ-YzAlzuaAvRJ2cXqyyadbsn_2S66' 
link1 = 'https://drive.google.com/open?id=1SAvSVsbqmVxmNXc1EaLo2VJKp4QTwpai'

In [5]:
#train data link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1EUwJ-YzAlzuaAvRJ2cXqyyadbsn_2S66


In [6]:
#test data link
fluff, id1 = link1.split('=')
print (id1) # Verify that you have everything after '='

1SAvSVsbqmVxmNXc1EaLo2VJKp4QTwpai


In [0]:
#train data
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('datahack_train.csv')  
data = pd.read_csv('datahack_train.csv')
# Dataset is now stored in a Panda

In [0]:
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('datahack_test.csv')  
data_test = pd.read_csv('datahack_test.csv')


In [0]:
data_test.drop('source',axis=1,inplace=True)

In [0]:
data_test.drop('financial_institution',axis=1,inplace=True)

In [0]:
data_test.drop('origination_date',axis=1,inplace=True)

In [0]:
data_test.drop('loan_purpose',axis=1,inplace=True)

In [0]:
data_test.drop('insurance_percent',axis=1,inplace=True)

In [0]:
data_test.drop('first_payment_date',axis=1,inplace=True)

In [15]:
X_test = data_test.iloc[:,1:9]
Y_test = data_test.loan_id
X_test.shape

(35866, 8)

In [0]:
data.drop('source',axis=1,inplace=True)

In [0]:
data.drop('financial_institution',axis=1,inplace=True)

In [0]:
data.drop('loan_purpose',axis=1,inplace=True)

In [0]:
data.drop('origination_date',axis=1,inplace=True)

In [0]:
data.drop('first_payment_date',axis=1,inplace=True)

In [0]:
data.drop('insurance_type',axis=1,inplace=True)

In [0]:
m = np.mean(data.insurance_percent)
n = format(m, '.2f')

In [0]:
def g(s):
  if(s==0):
    return(n)
  else:
    return(s)
data['insurance_percent'] = data.insurance_percent.apply(g)

In [0]:
data['unpaid_principal_bal'] = scl.fit_transform(data.unpaid_principal_bal.values.reshape(-1,1))

In [0]:
data.drop('number_of_borrowers',axis=1,inplace=True)

In [26]:
X = data.iloc[:,1:9]
X.shape

(116058, 8)

In [27]:
Y = data.m13
Y.value_counts()

0    115422
1       636
Name: m13, dtype: int64

In [28]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X,Y = smt.fit_resample(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from sklearn.model_selection import train_test_split as tts

In [0]:
xtrain,xtest,ytrain,ytest=tts(X,Y,test_size=0.4)

In [0]:
from sklearn.ensemble import AdaBoostClassifier

In [0]:
clf = AdaBoostClassifier()

In [33]:
clf.fit(xtrain,ytrain)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [34]:
clf.score(xtrain,ytrain)


0.8326281893925173

In [35]:
clf.score(xtest,ytest)

0.8299941519201196

In [0]:
pred = clf.predict(xtest)

In [0]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [38]:
accuracy_score(ytest,pred)

0.8299941519201196

In [39]:
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83     46135
           1       0.83      0.83      0.83     46203

    accuracy                           0.83     92338
   macro avg       0.83      0.83      0.83     92338
weighted avg       0.83      0.83      0.83     92338



In [0]:
predicted = clf.predict(X_test)

In [0]:
sub = pd.DataFrame()

In [0]:
sub['loan_id']=data_test['loan_id']
sub['m13']=predicted

In [0]:
pd.DataFrame(sub).to_csv('Mlhiring2.csv',index=False)

In [0]:
from google.colab import files

In [0]:
files.download('Mlhiring2.csv')